In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter  
import math 
from scipy import stats 

In [2]:
# Import NASDAQ list of stocks
stocks = pd.read_csv('nas.csv')
from secrets import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,AAPL
1,MSFT
2,AMZN
3,FB
4,TSLA
...,...
96,VRSN
97,DOCU
98,SWKS
99,SGEN


In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 97205397, 'calculationPrice': 'iexlasttrade', 'change': -0.817, 'changePercent': -0.00519, 'close': 0, 'closeSource': 'foficila', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 163.298, 'iexCloseTime': 1683783707485, 'iexLastUpdated': 1720858780317, 'iexMarketPercent': 0.01918898598414064, 'iexOpen': 163.63, 'iexOpenTime': 1668041417480, 'iexRealtimePrice': 168.7, 'iexRealtimeSize': 10, 'iexVolume': 2353816, 'lastTradeTime': 1716027930452, 'latestPrice': 159.046, 'latestSource': 'IEX Last Trade', 'latestTime': 'January 27, 2022', 'latestUpdate': 1717994410330, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2711512845522, 'oddLotDelayedP

In [4]:
pe_ratio = data['peRatio']
pe_ratio

14.85

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AAPL,162.813,14.72,N/A
1,MSFT,300.530,34.05,N/A
2,AMZN,2912.360,57.27,N/A
3,FB,302.680,21.67,N/A
4,TSLA,855.420,273.8,N/A


In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,59.0,3.3,N/A
1,WBA,52.0,7.07,N/A
2,SWKS,140.1,7.84,N/A
3,MRNA,152.7,9,N/A
4,INTC,50.3,9.69,N/A


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
portfolio_input()

Enter the value of your portfolio: 15000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,59.000,3.3,5
1,WBA,52.000,7.07,5
2,SWKS,140.100,7.84,2
3,MRNA,152.700,9,1
4,INTC,50.300,9.69,5
5,REGN,634.900,10.09,0
6,AMAT,131.920,10.23,2
7,QCOM,165.320,10.31,1
8,GILD,69.510,11.97,4
9,MU,82.250,12.72,3


In [10]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [11]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [12]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,165.828,N/A,14.44,N/A,21.29,N/A,3.71,N/A,11.558901,N/A,9.246768,N/A,N/A
1,MSFT,309.610,N/A,34.15,N/A,15.16,N/A,12.83,N/A,24.454231,N/A,17.761760,N/A,N/A
2,AMZN,2819.270,N/A,56.86,N/A,12.04,N/A,3.22,N/A,23.294216,N/A,7.603558,N/A,N/A
3,FB,295.770,N/A,21.64,N/A,6.56,N/A,7.74,N/A,14.061858,N/A,8.430818,N/A,N/A
4,TSLA,848.980,N/A,284.66,N/A,36.4,N/A,20.56,N/A,131.511420,N/A,85.995830,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,VRSN,208.850,N/A,38.43,N/A,-16.94,N/A,18.16,N/A,25.544694,N/A,21.192528,N/A,N/A
97,DOCU,110.960,N/A,-194,N/A,97.87,N/A,11.85,N/A,183.443452,N/A,14.544679,N/A,N/A
98,SWKS,143.030,N/A,7.7,N/A,2.23,N/A,2.4,N/A,7.683239,N/A,5.099912,N/A,N/A
99,SGEN,124.160,N/A,-70.49,N/A,7.28,N/A,13.03,N/A,-68.413375,N/A,13.892180,N/A,N/A


In [13]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [14]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.237624
1      0.623762
2      0.861386
3      0.336634
4      0.980198
         ...   
96     0.693069
97     0.019802
98     0.148515
99     0.069307
100    0.108911
Name: PE Percentile, Length: 101, dtype: object
0      0.861386
1      0.762376
2      0.693069
3      0.465347
4      0.910891
         ...   
96     0.059406
97     0.980198
98     0.188119
99     0.485149
100    0.990099
Name: PB Percentile, Length: 101, dtype: object
0      0.306931
1      0.811881
2      0.267327
3      0.574257
4      0.930693
         ...   
96     0.920792
97     0.762376
98     0.128713
99     0.821782
100    0.554455
Name: PS Percentile, Length: 101, dtype: object
0      0.287129
1      0.683168
2      0.653465
3      0.405941
4      0.950495
         ...   
96      0.70297
97     0.960396
98     0.128713
99     0.049505
100    0.089109
Name: EV/EBITDA Percentile, Length: 101, dtype: object
0      0.475248
1      0.821782
2      0.366337
3      0.455446
4           1.0
         ...   
9

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,165.828,N/A,14.44,0.237624,21.29,0.861386,3.71,0.306931,11.558901,0.287129,9.246768,0.475248,N/A
1,MSFT,309.610,N/A,34.15,0.623762,15.16,0.762376,12.83,0.811881,24.454231,0.683168,17.761760,0.821782,N/A
2,AMZN,2819.270,N/A,56.86,0.861386,12.04,0.693069,3.22,0.267327,23.294216,0.653465,7.603558,0.366337,N/A
3,FB,295.770,N/A,21.64,0.336634,6.56,0.465347,7.74,0.574257,14.061858,0.405941,8.430818,0.455446,N/A
4,TSLA,848.980,N/A,284.66,0.980198,36.40,0.910891,20.56,0.930693,131.511420,0.950495,85.995830,1.0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,VRSN,208.850,N/A,38.43,0.693069,-16.94,0.059406,18.16,0.920792,25.544694,0.70297,21.192528,0.861386,N/A
97,DOCU,110.960,N/A,-194.00,0.019802,97.87,0.980198,11.85,0.762376,183.443452,0.960396,14.544679,0.752475,N/A
98,SWKS,143.030,N/A,7.70,0.148515,2.23,0.188119,2.40,0.128713,7.683239,0.128713,5.099912,0.138614,N/A
99,SGEN,124.160,N/A,-70.49,0.069307,7.28,0.485149,13.03,0.821782,-68.413375,0.049505,13.892180,0.722772,N/A


In [15]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,165.828,N/A,14.44,0.237624,21.29,0.861386,3.71,0.306931,11.558901,0.287129,9.246768,0.475248,0.433663
1,MSFT,309.610,N/A,34.15,0.623762,15.16,0.762376,12.83,0.811881,24.454231,0.683168,17.761760,0.821782,0.740594
2,AMZN,2819.270,N/A,56.86,0.861386,12.04,0.693069,3.22,0.267327,23.294216,0.653465,7.603558,0.366337,0.568317
3,FB,295.770,N/A,21.64,0.336634,6.56,0.465347,7.74,0.574257,14.061858,0.405941,8.430818,0.455446,0.447525
4,TSLA,848.980,N/A,284.66,0.980198,36.40,0.910891,20.56,0.930693,131.511420,0.950495,85.995830,1.0,0.954455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,VRSN,208.850,N/A,38.43,0.693069,-16.94,0.059406,18.16,0.920792,25.544694,0.70297,21.192528,0.861386,0.647525
97,DOCU,110.960,N/A,-194.00,0.019802,97.87,0.980198,11.85,0.762376,183.443452,0.960396,14.544679,0.752475,0.69505
98,SWKS,143.030,N/A,7.70,0.148515,2.23,0.188119,2.40,0.128713,7.683239,0.128713,5.099912,0.138614,0.146535
99,SGEN,124.160,N/A,-70.49,0.069307,7.28,0.485149,13.03,0.821782,-68.413375,0.049505,13.892180,0.722772,0.429703


In [16]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:20]
rv_dataframe.reset_index(drop = True, inplace = True)

In [18]:
portfolio_input()

Enter the value of your portfolio: 15000


In [19]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\backup\envs\dev\envs\dev\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,WBA,51.000,14,6.91,0.138614,1.7000,0.128713,0.3403,0.009901,7.989597,0.138614,1.901444,0.009901,0.085149
1,SBUX,95.550,7,13.56,0.227723,-10.9900,0.069307,2.0200,0.113861,9.842060,0.227723,2.205542,0.019802,0.131683
2,SWKS,143.030,5,7.70,0.148515,2.2300,0.188119,2.4000,0.128713,7.683239,0.128713,5.099912,0.138614,0.146535
3,INTC,49.300,15,9.66,0.168317,2.3800,0.19802,2.7600,0.168317,5.994228,0.108911,4.888046,0.108911,0.150495
4,CMCSA,48.110,15,15.43,0.247525,2.3900,0.207921,2.0200,0.113861,8.427920,0.188119,2.705673,0.029703,0.157426
5,GILD,70.240,10,11.71,0.207921,4.1000,0.326733,3.1200,0.237624,8.001202,0.148515,4.964372,0.128713,0.209901
6,BIDU,147.470,5,23.30,0.425743,1.6300,0.118812,2.6700,0.148515,11.935094,0.306931,3.773802,0.049505,0.209901
7,MU,81.050,9,12.68,0.217822,2.0000,0.148515,3.2000,0.257426,5.889494,0.09901,7.267937,0.336634,0.211881
8,KHC,36.380,20,19.70,0.306931,0.9243,0.09901,1.7000,0.079208,10.605722,0.267327,7.177578,0.326733,0.215842
9,AMAT,131.340,5,10.33,0.19802,5.1300,0.386139,2.7100,0.158416,8.052194,0.158416,5.518205,0.19802,0.219802


In [ ]:
writer = pd.ExcelWriter('value_strat.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
writer.save()